: 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

: 

The question at hand: compute the steady-state "temperature" (value) in a square region whose boundary temperatures are known.
The Poisson eq. is written:

$$\frac{\partial^2{t(x,y)}}{\partial{x^2}}+\frac{\partial^2{t(x,y)}}{\partial{y^2}}=b(x,y)$$

In [ ]:
def xy(N):
    x = np.linspace( 0.0, L, N+2*ghost_zone ) 
    y = np.linspace( 0.0, L, N+2*ghost_zone )
    return x,y

: 

In [ ]:
# define a reference analytical solution
def u_ref(N):
    x = np.linspace( 0.0, L, N+2*ghost_zone ) 
    y = np.linspace( 0.0, L, N+2*ghost_zone )

    ref_func = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
    for i in range(N+2*ghost_zone):
        for j in range(N+2*ghost_zone):
            ref_func[i,j] = np.sin(x[i])*np.sin(y[j])+1
    return ref_func

#boundary condition
def source_func(N):
    exwhy = xy(N)
    rho = np.zeros((N+2*ghost_zone,N+2*ghost_zone))
    for i in range(N+2*ghost_zone):
        for j in range(N+2*ghost_zone):
            rho[i,j] = (-2*np.sin(exwhy[0][i])*np.sin(exwhy[1][j]))*(L/(N+1))**2 #multiplied by delta_x^2
    return rho

def initial_bound(N):
    if ghost_zone > 1:
        raise ValueError("Too big")
    ib = source_func(N)
    ur = u_ref(N)
    ib[ghost_zone,ghost_zone:-ghost_zone] -=  ur[0,ghost_zone:-ghost_zone] #top
    ib[-ghost_zone-1,ghost_zone:-ghost_zone] -=  ur[-1,ghost_zone:-ghost_zone]#bottom
    ib[ghost_zone:-ghost_zone,ghost_zone] -=  ur[ghost_zone:-ghost_zone,0]#left
    ib[ghost_zone:-ghost_zone,-ghost_zone-1] -=  ur[ghost_zone:-ghost_zone,-1] #right
    
    return ib[ghost_zone:-ghost_zone,ghost_zone:-ghost_zone]

: 

In [ ]:
N= 3
dimension = N**2

A = np.diagflat(np.ones(dimension)*-4, 0) + np.diagflat(np.ones(dimension-1)*1, 1) \
    + np.diagflat(np.ones(dimension-1)*1, -1) +np.diagflat(np.ones(dimension-N)*1, N) \
    + np.diagflat(np.ones(dimension-N)*1, -N)

b = initial_bound(N)[ghost_zone:-ghost_zone,ghost_zone:-ghost_zone].ravel()
#Use plt.imshow to see results
# print(u_ref(N)[ghost_zone:-ghost_zone,ghost_zone:-ghost_zone].ravel())
# print(A)
# print(b)
# print(u_ref(N))

: 

In [ ]:
#------------------------------------
#CG Solver
#------------------------------------
x = np.zeros((N**2, 1))  # first guess is zero vector
r = b  # r = b - A*x starts equal to b
d = r  # first search direction is r
while (np.dot(r.T, r)/N**4 > 10e-8) == True:  # still iterating
    alpha = np.dot(r.T, r) / np.dot(d.T, np.dot(A, d))
    x = x + alpha * d  # step to next guess
    rnew = r - alpha * np.dot(A, d)  # update residual r
    beta = np.dot(rnew.T, rnew) / np.dot(r.T, r) #correction
    r = rnew
    d = r + beta * d  # compute new search direction

: 